In [39]:
import pandas as pd 
import sqlite3

conn = sqlite3.connect("data/chinook.db")

In [67]:
pd.read_sql_query('''SELECT t.name as Song, g.name as Genre, a.title as Album,
ar.name as ArtistName, m.name as MediaTypes, t.unitprice as UnitPrice, t.Composer,
i.CustomerId, c.FirstName, c.LastName, ii.Quantity,
i.InvoiceDate,i.Billingcity as City,i.billingCountry as Country
from tracks t
left join playlist_track on playlist_track.trackid = t.name
left join playlists p on playlist_track.playlistid = p.playlistid
left join media_types m on m.mediatypeid = t.mediatypeid
left join genres g on g.genreid = t.genreid
left join albums a on a.albumid = t.albumid
left join artists ar on ar.artistid = a.artistid
left join invoice_items ii on ii.trackid = t.trackid
left join invoices i on ii.invoiceid = i.invoiceid
left join customers c on i.customerid = c.customerid''', conn)

,Song,Genre,Album,ArtistName,MediaTypes,UnitPrice,Composer,CustomerId,FirstName,LastName,Quantity,InvoiceDate,City,Country
0,For Those About To Rock (We Salute You),Rock,For Those About To Rock We Salute You,AC/DC,MPEG audio file,0.99,"Angus Young, Malcolm Young, Brian Johnson",47.0,Lucas,Mancini,1.0,2010-04-13 00:00:00,Rome,Italy
1,Balls to the Wall,Rock,Balls to the Wall,Accept,Protected AAC audio file,0.99,None,2.0,Leonie,Köhler,1.0,2009-01-01 00:00:00,Stuttgart,Germany
2,Balls to the Wall,Rock,Balls to the Wall,Accept,Protected AAC audio file,0.99,None,33.0,Ellie,Sullivan,1.0,2011-07-25 00:00:00,Yellowknife,Canada
3,Fast As a Shark,Rock,Restless and Wild,Accept,Protected AAC audio file,0.99,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",13.0,Fernanda,Ramos,1.0,2012-11-01 00:00:00,Brasília,Brazil
4,Restless and Wild,Rock,Restless and Wild,Accept,Protected AAC audio file,0.99,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",2.0,Leonie,Köhler,1.0,2009-01-01 00:00:00,Stuttgart,Germany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3756,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Classical,Schubert: The Late String Quartets & String Qu...,Emerson String Quartet,Protected AAC audio file,0.99,Franz Schubert,47.0,Lucas,Mancini,1.0,2010-04-13 00:00:00,Rome,Italy
3757,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Classical,Schubert: The Late String Quartets & String Qu...,Emerson String Quartet,Protected AAC audio file,0.99,Franz Schubert,13.0,Fernanda,Ramos,1.0,2012-11-01 00:00:00,Brasília,Brazil
3758,"L'orfeo, Act 3, Sinfonia (Orchestra)",Classical,Monteverdi: L'Orfeo,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...",Protected AAC audio file,0.99,Claudio Monteverdi,NaN,None,None,NaN,None,None,None
3759,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Classical,Mozart: Chamber Music,Nash Ensemble,Protected AAC audio file,0.99,Wolfgang Amadeus Mozart,NaN,None,None,NaN,None,None,None
